In [ ]:
# install dependencies: Zarr, Dask ML
!pip install -q zarr "dask-ml[complete]"

In [ ]:
# Import dependencies and load Sentinel 2 10m Composite Zarr
import gcsfs
import xarray as xr
import zarr
from dask.distributed import Client, progress
import dask_ml.cluster
import matplotlib.pyplot as plt

fs = gcsfs.GCSFileSystem(token='anon', access='read_only')
gcsmap = gcsfs.mapping.GCSMap("gs://cloud-geo-efm-public/s2-composite-10m/", gcs=fs, check=False, create=False)

# read the dataset
ds = xr.open_zarr(gcsmap)
ds

In [ ]:
# Crop the Dataset to the Manasquan, NJ Inlet area

bbox = [-74.09972442,  40.0838885 , -74.02481765,  40.12451048] # 500m buffer

ds_small = ds.sel(
    lat=slice(bbox[1], bbox[3]),
    lon=slice(bbox[0], bbox[2]),
    time='2022-01-01'
)

ds_small

In [ ]:
# Convert the DataSet to an Array

da = ds_small.to_array()

# stack lat and lon dimensions and transpose as KMeans expects input with a shape: (example, feature)
da = da.stack(point=['lat', 'lon']).transpose()
da

In [ ]:
# Start local Dask cluster with 1 worker and 4 threads per worker
client = Client(processes=False, threads_per_worker=4, n_workers=1, memory_limit='12GB')
client

In [ ]:
# Create 10 classes using Dask ML KNN on the local cluster using unsupervised classification

%%time
km = dask_ml.cluster.KMeans(n_clusters=10, init_max_iter=2, oversampling_factor=10)
km.fit(da)

In [ ]:
# Assign the predictions for each cell back to the DataSet

da['predicted_class'] = ('point', km.labels_)

# Unstack the data to restore the original lat/lon dimensions
da = da.unstack('point')
da

In [ ]:
# For reference, visualize Band 8 of the Sentinel 2 Composite

fig, ax = plt.subplots(figsize=(10, 5))
ds_small.B8.plot(x='lon',y='lat')

In [ ]:
# Plot the predicted classes

fig, ax = plt.subplots(figsize=(10, 5))
da.predicted_class.plot(ax=ax, add_colorbar=True, x='lon', y='lat', cmap='tab10_r')
ax.set_title("Unsupervised K-Means clustering on Zarr with Dask running locally")
plt.show()

In [ ]:
# Shutdown the Dask Client and Cluster
client.shutdown()